# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [2]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [3]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [4]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [5]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [13]:
gamma = 1
sigma = .005
r_=0.5/100

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .447 .


Pregunta 2:

Considere un swap a 1Y con cupones semestrales de 1.5% (lineal). Si la tasa
cupón cero a 0.5Y es 1.4% (exp), calcule la tasa cero a 1Y.


Pregunta 3:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 4:

Una opción sobre un bono cupón cero valorizada con el modelo de Vasicek vale lo
mismo que la misma opción valorizada con el modelo de Hull-White. Comente.


Pregunta 5:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 1282 y 2454 días.


Pregunta 6:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,022.00 (365 días después).


Pregunta 7:

Usando la curva 

## Respuestas

### Pregunta 1

El modelo de Vasicek para la tasa instantánea en la medida histórica es:

$$
\begin{equation}
dr_t=\gamma\left(\overline{r}-r_t\right)dt+\sigma dX_t
\end{equation}
$$

In [28]:
gamma=1
sigma=0.5/100
dt=1/264
r_=0.5/100
dXt=0.447
r0=curva(0)

drt = gamma * (r_-r0)*dt + sigma*dXt
print(f"El valor del primer paso de una simulación de MC para estos parámetros es: {drt: .8}")

El valor del primer paso de una simulación de MC para estos parámetros es:  0.0022508856


### Pregunta 2

### Pregunta 3

En los modelos estocásticos en general, al hacer un cambio de medida, lo que cambia en el modelo es el drift (tendencia), mientras que el coeficiente de difusión (volatilidad) se mantiene igual. De esta forma, en el modelo de Hull y White, este cambio de medida se ve reflejado en el theta de la dinámica de la tasa corta, que sería la única diferencia entre ambas medidas.

Así, ya en las valorizaciones, la principal diferencia entre valorizar por MC con la medida libre de riesgo y la medida T-forward es la manera en que se trae el payoff a valor presente. En el caso de la medida libre de riesgo, para encontrar el factor de descuento es necesario devolverse por cada trayectoria de las simulaciones, es decir, ir calculando un factor de descuento para cada step en que se discretizó el tiempo con cada tasa correspondiente al step, y así sucesivamente hasta llegar al presente. Recién ahí se puede utilizar el factor de descuento para traer a valor presente. En el caso de la medida forward esto no es así (lo cual es su principal ventaja), ya que el factor de descuento queda fuera de la esperanza, por lo que solo se debe utilizar la tasa correspondiente al instante de tiempo en el que se quiere valorizar, sin devolverse por cada trayectoria.


### Pregunta 4

No necesariamente es así. A pesar de que las fórmulas para Calls y Puts son iguales en ambos modelos, los valores de opciones con estos no son iguales. Dado que son opciones sobre bonos cero cupón, los valores de estas dependen del valor del bono. En primer lugar, las tasas simuladas con ambos modelos no son iguales, ya que en el modelo de HW se tiene una reversión a la media variable con el tiempo, mientras que en Vasicek esta es constante. Dado esto, el valor de los bonos de ambos modelos serán distintos, ya que dependen de las tasas, y por consiguiente el valor de la opción será distinto. Además, es importante mencionar que las fórmulas de calls y puts en ambos modelos dependen del valor del bono cero cupón a las fechas de madurez del bono en sí y de la opción. Los valores de los bonos se calculan de manera diferente en ambos modelos, difiriendo en el cálculo del A. Por último, dado que el drift de ambos modelos es distinto, la calibración de los parámetros no dará valores iguales para ambos modelos, lo que también influye en que el valor de las opciones sea distinto.

### Pregunta 5

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 1282 y 2454 días.

Sabemos que:

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$




Primero, es necesario interpolar para obtener el valor de la curva cero cupón entre 1282 y 2454 días. Después, se usa la fórmula de la tasa forward para obtenerla.

In [19]:
to=1282
t=2454
tasa_cero_cupon = curva(t-to)
tasa_cero_cupon

array(0.00085436)

In [18]:
dtasa_cero_cupon = (curva(t) - curva(to))/(t-to)    #aproximación de la derivada

In [22]:
tasa_forward = tasa_cero_cupon + (t-to)*dtasa_cero_cupon
print(f"El valor de la tasa forward entre 1282 y 2454 días es: {tasa_forward: .8}")

El valor de la tasa forward entre 1282 y 2454 días es:  0.00321888


### Pregunta 6



$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

Donde:

- TNA: significa *tasa nominal anual*.
- $ICP_{\tau}$: es el valor del $ICP$ la fecha $\tau=t,T$.
- $d$ es el número de días entre $t$ y $T$.
- $Round\left(x,4\right)$ redondea el número $x$ a 4 decimales.

In [38]:
ICPo= 10000
ICPT = 10022

TNA = round(((ICPT/ICPo)-1) * 360/365, 4)
print(f"El valor de la TNA 365 días después es: {TNA: .8}")

El valor de la TNA 365 días después es:  0.0022


### Pregunta 7

In [47]:
c_p = hw.CallPut.CALL
K = 98/100
r0= curva(0)
TO=1
TB=2
dfTO = math.exp(-curva(TO)*TO)  #cálculo del factor de descuento de mercado al tiempo TO mediante la tasa interpolada de la curva
dfTB = math.exp(-curva(TB)*TB)  #cálculo del factor de descuento de mercado al tiempo TB mediante la tasa interpolada de la curva
gamma=1
sigma=0.5/100

# AD: ¡los plazos de la curva están en días!

In [50]:
valor_call = hw.zcb_call_put(c_p, K, r0, TO, TB, dfTO, dfTB, gamma, sigma)
print(f"El valor de la Call con un nocional de 1 y los parámetros dados es: {valor_call: .8}")

El valor de la Call con un nocional de 1 y los parámetros dados es:  0.019163877
